# Выполнение задания по LSTM

## Пробуем создать и обучить модель на базе LSTM

In [11]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras import layers # type: ignore (чтобы жёлтым не подчёркивало)

# Сразу же подготавливаем наши файлы (store.csv | train.csv)
train = pd.read_csv('train.csv', low_memory=False)
store = pd.read_csv('store.csv')
# Объединяем данные по идентификатору магазина
data = pd.merge(train, store, on='Store', how='left')
print(data.columns)
# data.head(10)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')


In [12]:
# Подготавливаем признаки (извлекаем год, месяц, день | удаляем дату)
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data = data.drop(columns=['Date'])

# Удаляем ненужные строки и столбцы
data = data[data['Open'] == 1]
data = data.drop(columns=['Open', 'Customers'])

# Заполняем пропуски значениями для корректного обучения
data['CompetitionDistance'] = data['CompetitionDistance'].fillna(data['CompetitionDistance'].median())
data['PromoInterval'], _ = pd.factorize(data['PromoInterval'])
data = data.fillna(0)

# Преобразуем категориальные переменные в числовые
data = pd.get_dummies(data, columns=['DayOfWeek', 'StoreType', 'Assortment', 'StateHoliday'], drop_first=True)

# Преобразуем логические значения в целочисленные
for column in data.select_dtypes(include=['bool']).columns:
    data[column] = data[column].astype(int)

In [13]:
# Далее разбиваем данные и подготавливаем последовательности
train_size = int(len(data) * 0.05)
test_size = int(len(data) * 0.05)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Функция создания последовательностей
def create_sequences(X, y, time_steps=14):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# разделяем данные на признаки и целевую переменную
X_train = train_data.drop(columns=['Sales']).values
y_train = train_data['Sales'].values
X_test = test_data.drop(columns=['Sales']).values
y_test = test_data['Sales'].values

In [14]:
# Нормализуем данные через StandartScaler
# Делаем так, чтобы все признаки имели одинаковый масштаб
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создаём последовательности данных длинной в 2 недели
time_steps = 14
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)

In [16]:
# Создаём модельку
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(64, activation='relu', return_sequences=True))
model.add(layers.LSTM(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 14, 64)              │          23,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,001 (140.63 KB)

 Trainable params: 36,001 (140.63 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Обучение модели
# Стоит переключиться на GPU
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

Epoch 1/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - loss: 20135584.0000 - val_loss: 7318919.0000
Epoch 2/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - loss: 7013385.0000 - val_loss: 6643561.5000
Epoch 3/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - loss: 6863786.0000 - val_loss: 6341965.0000
Epoch 4/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 6406606.0000 - val_loss: 6006550.0000
Epoch 5/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - loss: 5829451.5000 - val_loss: 6675170.5000
Epoch 6/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - loss: 5522030.0000 - val_loss: 5508163.5000
Epoch 7/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - loss: 4998227.0000 - val_loss: 5552277.0000
Epoch 8/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - loss: 4107240.2500 - val_loss: 3966975.2500
Epoch 9/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 3357031.2500 - val_loss: 3131206.7500
Epoch 10/50
594/594 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 2668907.5000 - val_loss: 2941707.750

In [18]:
# Замечаем, что ошибка получилась не очень б_б
# Прописываем функцию для предсказывания и визуализации результатов
def plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len):
    current_start = 0
    plt.figure(figsize=(12, 10))

    while current_start < len(X_test_scaled):
        end_idx = current_start + test_size
        if end_idx >= len(X_test_scaled):
            end_idx = len(X_test_scaled)

        X_test_seq, y_test_seq = create_sequences(X_test_scaled[current_start:end_idx], y_test[current_start:end_idx], time_steps)
        y_test_pred = model.predict(X_test_seq)

        plt.figure(figsize=(12, 6))
        plt.plot(y_test_seq, label='Настоящие значения')
        plt.plot(y_test_pred, label='Предсказанные значения')
        plt.legend()
        plt.title(f'Предсказания с {current_start} по {end_idx}')
        plt.show()

        current_start += test_size

In [ ]:
# Вызываем нашу функцию и результаты
total_data_len = len(data)
plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len)
# Можно абалдеть от того сколько времени будут строиться наши предсказания

## Самостоятельная работа

### 1. Изменить количество временных шагов (timestamps)

```python
# Пример изменения time_steps
time_steps = 7
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)
```

Или можно сразу прописать изменение в функции

```python
def create_sequences(data, time_steps=14):
    sequences, labels = [], []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:i+time_steps])
        labels.append(data[i+time_steps])
    return np.array(sequences), np.array(labels)
```


Если мы уменьшим количество дней, то можно заметить, что ошибка значительно уменьшится, что логично, так как предсказания будут строиться на меньшее кол-во дней. Если же наоборот увеличить количество дней, то ошибка значительно возрастёт и предсказания уже не будут такими точными

p.s. Ну, по крайней мере у меня так)

### 2. Изменить количество слоёв в модели

#### Ожидаемые эффекты
Меньшее количество нейронов:

+ Уменьшение количества нейронов может ускорить обучение и уменьшить объём ресурсов, требуемых для расчёта.
+ Однако модель может потерять способность захватывать сложные зависимости, и точность прогноза может снизиться.

Большее количество нейронов:

+ Увеличение количества нейронов может улучшить способность модели выявлять более сложные паттерны, особенно в случае временных рядов с большим количеством данных.
+ Но это также может привести к переобучению, если количество данных недостаточно, и модель начинает подстраиваться под шум.
+ Большее количество нейронов увеличивает вычислительную сложность и время обучения, что нужно учитывать.

#### Что у меня получилось заметить:

Увеличив кол-во нейронов модель (возможно мне показалась) стала медленнее обучаться. Увеличился коэф-ент ошибки и в целом, модель начала переобучаться. Понял я это так как ошибка стала постоянно скакать от меньшего к большему, а потом и вовсе стала возрастать, что является первым признаком переобучения.

Если попробовать уменьшить нейронов в модели, то ошибка снова возрастёт, а сама эффективность модели (предсказательные возмости) станут хуже.

В целом, за счёт добавления новых слоёв и оптимизации обучения модели можно будет добиться идеального результата

### 3. Добавить слой Dropout для борьбы с переобучением

#### Ожидаемые эффекты:
+ Улучшение обобщающей способности: Dropout заставляет модель полагаться на все нейроны, что способствует более «устойчивому» обучению.
+ Увеличение времени обучения: Поскольку каждый шаг будет обучаться с неполным набором активных нейронов, это может замедлить обучение, но часто улучшает точность на тестовых данных.

#### Что у меня получилось заметить:

Модель не стала дольше обучаться. Возможно сама архитектура задачи не сильно реагирует на изменение с помощью dropout. На каждую итерацию уходило по 2 - 3 секунды. Мне кажется, что модель наоборот стала обучаться быстрее, особенно это заметно если сравнивать пример из материалов урока и нашу модель.

Мной не было замечено никаких минусов. Думаю что в данной задаче можно смело использовать dropout. Ошибка никак не изменилась, выходные результаты будто бы тоже.

*Ещё хотел бы отметить, что в модели значительно увеличилось кол-во параметров*

### 4. Добавить больше слоев LSTM

#### Ожидаемые эффекты
- Большее количество слоев LSTM может повысить точность модели на сложных данных, если в данных присутствуют длительные зависимости.
- Риск переобучения: Слишком много слоев может привести к переобучению, особенно если количество данных ограничено.
- Возросшая вычислительная сложность: Добавление слоев увеличивает количество параметров, что потребует больше вычислительных ресурсов.

#### Что у меня получилось заметить:

Заметно, что сам процесс обучения у модели стал проходить труднее (если можно так выразиться), так как процент ошибки стал падать медленнее. Также увеличилось время, необходимое для полного обучения модели. Если в dropout на обучение одной эпохи уходило 2 секунды, сейчас уходит 5 (без dropout)

Я пробовал использовать такую конфигурацию:

```python
# Модель с дополнительными слоями LSTM
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='relu', return_sequences=True))
model.add(layers.LSTM(64, activation='relu', return_sequences=True))  # Добавлен еще один LSTM слой
model.add(layers.LSTM(32, activation='relu'))  # Новый заключительный LSTM слой
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()
```

### 5. Изменить функцию активации на других слоях

#### Ожидаемые эффекты
- Изменение динамики обучения: Активные функции влияют на скорость и стабильность обучения, и иногда tanh или sigmoid могут лучше выявлять зависимости, чем ReLU.
- Лучшее качество предсказаний: В зависимости от типа данных функция активации может либо улучшить точность модели, либо замедлить её обучение, так что важно экспериментировать и сравнивать.

В моём случае обе функции замедлили процесс обучения и значительно увеличили ошибку

```python
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='tanh', return_sequences=True))  # Изменена активация на tanh
model.add(layers.LSTM(64, activation='tanh'))  # Также изменена активация на tanh
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()
```


```python
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='sigmoid', return_sequences=True))  # Изменена активация на tanh
model.add(layers.LSTM(64, activation='sigmoid'))  # Также изменена активация на tanh
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()
```

### 6. Использовать больше данных для обучения

Я попробовал выделить 10 и 20 процентов данных для обучения модели. Заметил, что при увеличении данных значительно увеличивается время обучения. Для модели с dropout и выделенными 10 процентами для обучения, время обучения одной эпохи увеличелось с 2 секунд до 5 - 10 секунд. Процент ошибки незначительно возрос. Но, на выходные данные это особо не повлияло XD

### 7. Изменить размер батча

Ну и заключительное задание, связанное с изменением размера пакетов. Если увеличивать размер пакетов, то скорость обучения значительно возрастает, однако падает эффективность обучения модели (соответственно возрастает ошибка)

Как раз на выходных данных видно наксколько сильно меняются результаты предсказания.

## Тестируем и балуемся

In [14]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras import layers # type: ignore (чтобы жёлтым не подчёркивало)

# Сразу же подготавливаем наши файлы (store.csv | train.csv)
train = pd.read_csv('train.csv', low_memory=False)
store = pd.read_csv('store.csv')
# Объединяем данные по идентификатору магазина
data = pd.merge(train, store, on='Store', how='left')
print(data.columns)
# data.head(10)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')


In [15]:
# Подготавливаем признаки (извлекаем год, месяц, день | удаляем дату)
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data = data.drop(columns=['Date'])

# Удаляем ненужные строки и столбцы
data = data[data['Open'] == 1]
data = data.drop(columns=['Open', 'Customers'])

# Заполняем пропуски значениями для корректного обучения
data['CompetitionDistance'] = data['CompetitionDistance'].fillna(data['CompetitionDistance'].median())
data['PromoInterval'], _ = pd.factorize(data['PromoInterval'])
data = data.fillna(0)

# Преобразуем категориальные переменные в числовые
data = pd.get_dummies(data, columns=['DayOfWeek', 'StoreType', 'Assortment', 'StateHoliday'], drop_first=True)

# Преобразуем логические значения в целочисленные
for column in data.select_dtypes(include=['bool']).columns:
    data[column] = data[column].astype(int)

In [26]:
# Далее разбиваем данные и подготавливаем последовательности
train_size = int(len(data) * 0.05)
test_size = int(len(data) * 0.05)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Функция создания последовательностей
def create_sequences(X, y, time_steps=14):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# разделяем данные на признаки и целевую переменную
X_train = train_data.drop(columns=['Sales']).values
y_train = train_data['Sales'].values
X_test = test_data.drop(columns=['Sales']).values
y_test = test_data['Sales'].values

In [27]:
# Нормализуем данные через StandartScaler
# Делаем так, чтобы все признаки имели одинаковый масштаб
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создаём последовательности данных длинной в 2 недели
time_steps = 7
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)

In [28]:
# Создание модели с добавлением Dropout
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.2))  # Dropout после первого LSTM слоя
model.add(layers.LSTM(64, activation='relu'))
model.add(layers.Dropout(0.2))  # Dropout после второго LSTM слоя
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 7, 128)              │          79,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 7, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 129,345 (505.25 KB)

 Trainable params: 129,345 (505.25 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Модель с дополнительными слоями LSTM
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='relu', return_sequences=True))
model.add(layers.LSTM(64, activation='relu', return_sequences=True))  # Добавлен еще один LSTM слой
model.add(layers.LSTM(32, activation='relu'))  # Новый заключительный LSTM слой
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 7, 128)              │          79,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 7, 64)               │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 141,729 (553.63 KB)

 Trainable params: 141,729 (553.63 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Модель с изменённой функцией активации
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='tanh', return_sequences=True))  # Изменена активация на tanh
model.add(layers.LSTM(64, activation='tanh'))  # Также изменена активация на tanh
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 7, 128)              │          79,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 129,345 (505.25 KB)

 Trainable params: 129,345 (505.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(128, activation='sigmoid', return_sequences=True))
model.add(layers.LSTM(64, activation='sigmoid'))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 7, 128)              │          79,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 129,345 (505.25 KB)

 Trainable params: 129,345 (505.25 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Обучение модели
# Стоит переключиться на GPU
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=512, validation_split=0.1, verbose=1)

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 71ms/step - loss: 47777448.0000 - val_loss: 7814581.0000
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8233192.0000 - val_loss: 7413561.0000
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7490632.0000 - val_loss: 7115528.0000
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7372843.0000 - val_loss: 6958172.5000
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7470914.5000 - val_loss: 6701439.5000
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7099585.0000 - val_loss: 6634939.0000
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6771191.0000 - val_loss: 6565932.0000
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6906929.0000 - val_loss: 6421429.0000
Epoch 9/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6701520.5000 - val_loss: 6291823.5000
Epoch 10/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6513465.5000 - val_loss: 6115175.0000
Epoch 11/50
75/75 ━━━━━━━━━━━━━━━━━━

In [30]:
# Замечаем, что ошибка получилась не очень б_б
# Прописываем функцию для предсказывания и визуализации результатов
def plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len):
    current_start = 0
    plt.figure(figsize=(12, 10))

    while current_start < len(X_test_scaled):
        end_idx = current_start + test_size
        if end_idx >= len(X_test_scaled):
            end_idx = len(X_test_scaled)

        X_test_seq, y_test_seq = create_sequences(X_test_scaled[current_start:end_idx], y_test[current_start:end_idx], time_steps)
        y_test_pred = model.predict(X_test_seq)

        plt.figure(figsize=(12, 6))
        plt.plot(y_test_seq, label='Настоящие значения')
        plt.plot(y_test_pred, label='Предсказанные значения')
        plt.legend()
        plt.title(f'Предсказания с {current_start} по {end_idx}')
        plt.show()

        current_start += test_size

In [ ]:
# Вызываем нашу функцию и результаты
total_data_len = len(data)
plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len)
# Можно абалдеть от того сколько времени будут строиться наши предсказания